In [1]:
import pandas as pd 
import glob
import cv2 as cv
import random
import os

import matplotlib.pyplot as plt
import numpy as np
import random
from PIL import Image
import PIL.ImageOps    

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import torchvision.utils
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from sklearn.metrics import f1_score, accuracy_score, classification_report, recall_score, precision_score

from tqdm.auto import tqdm
import timm
import math
from sklearn.model_selection import train_test_split

import segmentation_models_pytorch as smp
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

c:\Users\EHmin\anaconda3\envs\EHmin\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = 'a'
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

CFG = {
    'IMG_SIZE':224,
    'EPOCHS':20,
    'LEARNING_RATE':3e-4,
    # 'LEARNING_RATE':10,
    'BATCH_SIZE':8,
    'SEED':41
}

seed_everything(CFG['SEED']) # Seed 고정

In [3]:
# train_target = glob.glob("../Data/train_target_image/*")
# train_target.sort()
# train_target

In [4]:
# template = cv.imread(train_target[1])
# template = template[908:1074, 136:428]


In [5]:
# img2 = cv.imread(train_target[44])
# result = cv.matchTemplate(img2, template, cv.TM_CCOEFF_NORMED)
# threshold = 0.5
# locations = np.where(result >= threshold)
# if locations[0].size > 0:
#     print("있어")
# else:
#     print("없어")


In [6]:
# ans = []

# for img_path in tqdm(train_target):
#     img = cv.imread(img_path)
#     result = cv.matchTemplate(img, template, cv.TM_CCOEFF_NORMED)
#     threshold = 0.8
#     locations = np.where(result >= threshold)
#     if locations[0].size > 0:
#         ans.append(0) #0 이면 뒤
#     else:
#         ans.append(1) # 1이면 앞

    

In [7]:
# df = pd.DataFrame(columns=['image','label'])
# df['image'] = train_target
# df['label'] = ans
# df.to_csv('my_data.csv', index=False)


In [8]:
df = pd.read_csv('./my_data.csv', )
df

,image,label
0,../Data/train_target_image\TRAIN_TARGET_0000.png,1
1,../Data/train_target_image\TRAIN_TARGET_0001.png,0
2,../Data/train_target_image\TRAIN_TARGET_0002.png,1
3,../Data/train_target_image\TRAIN_TARGET_0003.png,1
4,../Data/train_target_image\TRAIN_TARGET_0004.png,1
...,...,...
2918,../Data/train_target_image\TRAIN_TARGET_2918.png,0
2919,../Data/train_target_image\TRAIN_TARGET_2919.png,1
2920,../Data/train_target_image\TRAIN_TARGET_2920.png,1
2921,../Data/train_target_image\TRAIN_TARGET_2921.png,1


In [9]:
class CustomDataset(Dataset):
    def __init__(self, image, label, transform=None, infer=False):
        self.source = image
        self.gt = label
        self.transform = transform
        self.infer = infer


    def __getitem__(self, idx):
        img_path = self.source[idx]
        image = cv.imread(img_path)
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
        
        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image
        
        label = self.gt[idx]

        if self.transform: # 알부네이션 먹이이는 형식으로 진행 
            augmented = self.transform(image=image) 
            image = augmented['image']
            
        return image, label
    
    def __len__(self):
        return len(self.source)
    

In [10]:
transform = A.Compose(
    [   
        A.Resize(224, 224),
        A.Normalize(),
        ToTensorV2()
    ]
)

In [11]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.2, random_state=CFG['SEED'])

In [12]:
train_dataset = CustomDataset(image = train['image'].values, label = train['label'].values, transform=transform, infer=False)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0)

val_dataset = CustomDataset(image = val['image'].values, label = val['label'].values, transform=transform, infer=False)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True, num_workers=0)

In [13]:
class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()
        self.backbone = timm.create_model('efficientnet_b7', pretrained=False)
        self.classifier = nn.Linear(1000, 2)
        self.dropout = nn.Dropout(0.1)
        self.ReLU = nn.ReLU(inplace=False)
        self.classifier2 = nn.Linear(50, 2)
    
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        
        return F.log_softmax(x, dim=1)
    
        
    

In [14]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []
  
    
    with torch.no_grad():
        for landmark_list, labels in tqdm(iter(val_loader)):
            
            landmark_list = landmark_list.float().to(device)
            
            labels = labels.type(torch.LongTensor).to(device)
            
            pred = model(landmark_list)

            loss = criterion(pred, labels)
            
            preds += pred.detach().argmax(1).cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
            # print(preds, true_labels)
            # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html
            
        _val_loss = np.mean(val_loss)
        _val_score_f1 = f1_score(true_labels, preds, average='weighted')
        _val_score_acc = accuracy_score(true_labels, preds)
        _val_score_recall = recall_score(true_labels, preds, average='macro')
        _val_score_precision = precision_score(true_labels, preds, average='macro')
        report = classification_report(true_labels, preds)
        
        # print(accuracy_score(true_labels, preds))
        
        
    
    return _val_loss, _val_score_f1, _val_score_acc, report , _val_score_recall, _val_score_precision

In [15]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model = model.to(device)
    criterion = nn.NLLLoss(weight=torch.tensor([0.01, 0.99]), reduction="sum").to(device)

    best_score = 0
    best_model = None
    
    for epoch in range(0, CFG['EPOCHS']):
        model.train()
        train_loss = []
        
        for landmark_list, labels in tqdm(iter(train_loader)):
            
            landmark_list = landmark_list.float().to(device)
            
            labels = labels.type(torch.LongTensor).to(device)
            
            optimizer.zero_grad()
            output = model(landmark_list)
            
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
                    
                
        _val_loss, _val_score_f1, _val_score_acc, report , _val_score_recall, _val_score_precision = validation(model, criterion, val_loader, device)
        
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val accuracy score : [{_val_score_acc:.5f}] Val recall score : [{_val_score_recall:.5f}] Val f1 score : [{_val_score_f1:.5f}]')
        print(report)
        if scheduler is not None:
            scheduler.step(_val_score_recall)
            
        if best_score < _val_score_acc:
            best_score = _val_score_acc
            best_model = model
            torch.save(best_model.state_dict(), "./frontOrBack.pt")
            
            
    
    return best_model

In [16]:
model = BaseModel()

In [17]:
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

100%|██████████| 37/37 [00:31<00:00,  1.16it/s]
c:\Users\EHmin\anaconda3\envs\EHmin\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\EHmin\anaconda3\envs\EHmin\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\EHmin\anaconda3\envs\EHmin\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, 

Epoch [0], Train Loss : [0.51962] Val Loss : [0.28363] Val accuracy score : [0.53846] Val recall score : [0.50000] Val f1 score : [0.37692]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       270
           1       0.54      1.00      0.70       315

    accuracy                           0.54       585
   macro avg       0.27      0.50      0.35       585
weighted avg       0.29      0.54      0.38       585



100%|██████████| 37/37 [00:29<00:00,  1.25it/s]
c:\Users\EHmin\anaconda3\envs\EHmin\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\EHmin\anaconda3\envs\EHmin\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\EHmin\anaconda3\envs\EHmin\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, 

Epoch [1], Train Loss : [0.23199] Val Loss : [0.21896] Val accuracy score : [0.53846] Val recall score : [0.50000] Val f1 score : [0.37692]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       270
           1       0.54      1.00      0.70       315

    accuracy                           0.54       585
   macro avg       0.27      0.50      0.35       585
weighted avg       0.29      0.54      0.38       585



100%|██████████| 37/37 [00:29<00:00,  1.26it/s]


...

Epoch [17], Train Loss : [0.00988] Val Loss : [0.17810] Val accuracy score : [0.98974] Val recall score : [0.98942] Val f1 score : [0.98974]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       270
           1       0.99      0.99      0.99       315

    accuracy                           0.99       585
   macro avg       0.99      0.99      0.99       585
weighted avg       0.99      0.99      0.99       585



100%|██████████| 37/37 [00:30<00:00,  1.21it/s]


Epoch [18], Train Loss : [0.00448] Val Loss : [0.10274] Val accuracy score : [0.98974] Val recall score : [0.98915] Val f1 score : [0.98974]
              precision    recall  f1-score   support

           0       1.00      0.98      0.99       270
           1       0.98      1.00      0.99       315

    accuracy                           0.99       585
   macro avg       0.99      0.99      0.99       585
weighted avg       0.99      0.99      0.99       585



100%|██████████| 37/37 [00:30<00:00,  1.22it/s]

Epoch [19], Train Loss : [0.00270] Val Loss : [0.13165] Val accuracy score : [0.98632] Val recall score : [0.98545] Val f1 score : [0.98631]
              precision    recall  f1-score   support

           0       1.00      0.97      0.99       270
           1       0.98      1.00      0.99       315

    accuracy                           0.99       585
   macro avg       0.99      0.99      0.99       585
weighted avg       0.99      0.99      0.99       585



In [36]:
#폴더 이동시 경로 수정이 필요할 수 있음 
test_dataset = glob.glob("../Data/test_image/*")

# glob 이후에 정렬이 안되어 있기 때문에, source - gt matching을 위해 정렬
test_dataset.sort()

print(test_dataset)

['../Data/test_image\\TEST_0000.png', '../Data/test_image\\TEST_0001.png', '../Data/test_image\\TEST_0002.png', '../Data/test_image\\TEST_0003.png', '../Data/test_image\\TEST_0004.png', '../Data/test_image\\TEST_0005.png', '../Data/test_image\\TEST_0006.png', '../Data/test_image\\TEST_0007.png', '../Data/test_image\\TEST_0008.png', '../Data/test_image\\TEST_0009.png', '../Data/test_image\\TEST_0010.png', '../Data/test_image\\TEST_0011.png', '../Data/test_image\\TEST_0012.png', '../Data/test_image\\TEST_0013.png', '../Data/test_image\\TEST_0014.png', '../Data/test_image\\TEST_0015.png', '../Data/test_image\\TEST_0016.png', '../Data/test_image\\TEST_0017.png', '../Data/test_image\\TEST_0018.png', '../Data/test_image\\TEST_0019.png', '../Data/test_image\\TEST_0020.png', '../Data/test_image\\TEST_0021.png', '../Data/test_image\\TEST_0022.png', '../Data/test_image\\TEST_0023.png', '../Data/test_image\\TEST_0024.png', '../Data/test_image\\TEST_0025.png', '../Data/test_image\\TEST_0026.png', 

In [25]:
df_test = pd.DataFrame(columns=['test'])
df_test['test'] = test_dataset

df_test

,test
0,../Data/test_image\TEST_0000.png
1,../Data/test_image\TEST_0001.png
2,../Data/test_image\TEST_0002.png
3,../Data/test_image\TEST_0003.png
4,../Data/test_image\TEST_0004.png
...,...
1893,../Data/test_image\TEST_1893.png
1894,../Data/test_image\TEST_1894.png
1895,../Data/test_image\TEST_1895.png
1896,../Data/test_image\TEST_1896.png


In [27]:
test_dataset = CustomDataset(image = df_test['test'].values ,label = _ , transform=transform, infer=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=0 )

In [33]:
with torch.no_grad():
    infer_model.eval()
    result = []
    for images in tqdm(test_loader):
        images = images.float().to(device)
        pred = infer_model(images)
        result += pred.detach().argmax(1).cpu().numpy().tolist()
            

100%|██████████| 119/119 [01:38<00:00,  1.21it/s]


In [37]:
df = pd.DataFrame(columns=['source','label'])
df['source'] =  test_dataset
df['label'] = result
df

,source,label
0,../Data/test_image\TEST_0000.png,1
1,../Data/test_image\TEST_0001.png,1
2,../Data/test_image\TEST_0002.png,1
3,../Data/test_image\TEST_0003.png,1
4,../Data/test_image\TEST_0004.png,1
...,...,...
1893,../Data/test_image\TEST_1893.png,0
1894,../Data/test_image\TEST_1894.png,0
1895,../Data/test_image\TEST_1895.png,1
1896,../Data/test_image\TEST_1896.png,0


In [38]:
df.to_csv('./front_OR_back.csv', index=False)
# 0 이 back 1이 front